# Multiple simulations of the MARM1 model

This notebook allows to run multiple sequential simulations of the MARM1 model in order to generate a dataset for multiple conditions. For a detailed description of how a single simulation run is set up and the visulization of a time-course trajectory please refer to the Jupyter Notebook: *MARM1_simulation_single_run.ipynb*. 

Importing the model and libraries necessary to run MARM1 model simulations. 

In [61]:
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#load model
from pysb.simulator import ScipyOdeSimulator
from pysb.core import as_complex_pattern
from MARM1 import model

Generating the condition settings (e.g. RAFi and MEKi concentration) for each model simulations to be run.

In [62]:
#define conditions
RAFi_concentration = np.logspace(-4, 1, 11)
MEKi_concentration = np.logspace(-5, 0, 11)
t_pretrt = [24]
EGF_concentration = [100]
t_trt = [2]
param_set_index = [0]
N_time_points = [100]
#generate condition settings for multiple simulations
settings_list=[]
for rafi in RAFi_concentration:
    for meki in MEKi_concentration:
        for pretrt in t_pretrt:
            for trt in t_trt:
                for egfc in EGF_concentration:
                    for param in param_set_index:
                        settings_list.append([param, pretrt, rafi, meki, trt, egfc])

Defining support functions to run simulations. 

In [63]:
def equilibrate(simulator, initials):
    """Simulate a model from given initial conditions until it reaches steady state"""
    scale = 10
    t_start = 1e-4
    df = None
    tspan = np.geomspace(t_start, t_start * scale)
    while True:
        print(f"    at t={tspan[-1]:<5.3g} ... ", end='', flush=True)
        res = simulator.run(tspan=tspan, initials=initials)
        df = pd.concat([df, res.dataframe.iloc[1:]])
        initials = res.species[-1]
        close = np.isclose(
            *res.species[[-1,-2]].view(float).reshape(2,-1),
            rtol=1e-3
        )
        cs = np.sum(close)
        n = len(model.species)
        print(f"{cs}/{n} species converged")
        if np.all(close):
            break
        tspan *= scale
    return df

In [64]:
def get_species_index(model, pattern):
    """Return the integer species number for a given species in the model"""
    pattern = as_complex_pattern(pattern)
    matches = [
        i for i, s in enumerate(model.species)
        if s.is_equivalent_to(pattern)
    ]
    n = len(matches)
    assert n == 1, f"Expected exactly one match, got {n}"
    return matches[0]

Simulating multiple condition runs and save time-course trajectories on disk. 

In [ ]:
#to be continued.. 